In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol

using CairoMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances
using BenchmarkTools

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [53]:
function get_flat_realization(x, template_mol)
    n_mol = length(x) ÷ 6
    [(hvcat((n_mol), [exp(Rotations.RotationVecGenerator(x[i:i+2]...)) * template_mol .+ x[i+3:i+5] for i in 1:6:length(x)]...)...)...]
end

get_flat_realization (generic function with 1 method)

In [88]:
function rotation_and_translation_gradient!(∇E, x, ∇FSol, template_mol)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x) ÷ 6
    for i in 1:n_mol        
        R = exp(Rotations.RotationVecGenerator(x[(i-1)*6 + 1:(i-1)*6 + 3]...))
        ∇E[(i-1) * 6 + 1] = 0.5 * sum([-v[2]*(R[3,:] ⋅ w) + v[3]*(R[2,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 2] = 0.5 * sum([v[1]*(R[3,:] ⋅ w) - v[3]*(R[1,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 3] = 0.5 * sum([-v[1]*(R[2,:] ⋅ w) + v[2]*(R[1,:] ⋅ w) for (v,w) in [(∇FSol[:,:,i][:,j], template_mol[:,j]) for j in 1:n_atoms_per_mol]])
        ∇E[(i-1) * 6 + 4:(i-1) * 6 + 6] = sum([∇FSol[:,j,i] for j in 1:n_atoms_per_mol])
    end
    ∇E
end

function solvation_free_energy_gradient!(∇E, x, template_mol, radii, rs, pf, overlap_slope)
    n_atoms_per_mol = size(template_mol)[2]
    n_mol = length(x) ÷ 6
    flat_realization = get_flat_realization(x, template_mol)
    _, dvol, dsurf, dmean, dgauss, dlol = MorphoMol.Energies.get_geometric_measures_and_overlap_value_with_derivatives(
        flat_realization,
        n_atoms_per_mol,
        radii,
        rs,
        0.0,
        overlap_slope
    )
    ∇FSol = reshape(pf[1] * dvol + pf[2] * dsurf + pf[3] * dmean + pf[4] * dgauss + dlol, (3, n_atoms_per_mol, n_mol))
    rotation_and_translation_gradient!(∇E, x, ∇FSol, template_mol)
end


solvation_free_energy_gradient! (generic function with 1 method)

In [89]:
function solvation_free_energy(x::Vector{Float64}, template_mol::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, delaunay_eps::Float64)
    n_mol = length(x) ÷ 6
    n_atoms_per_mol = size(template_mol)[2]
    flat_realization = get_flat_realization(x, template_mol)
    MorphoMol.Energies.solvation_free_energy(flat_realization, n_atoms_per_mol, radii, rs, prefactors, overlap_jump, overlap_slope, delaunay_eps)
end

solvation_free_energy (generic function with 2 methods)

In [90]:
n_mol = 20
x_init = vcat([[rand(3) * pi; rand(3) * 10.0] for i in 1:n_mol]...)

a = 2.0
#template_mol = [0.0, 0.0, 0.0, 2*a, 0.0, 0.0, a, sqrt(3)*a, 0.0]
template_mol = [0.0, 0.0, 0.0, a, 0.0, 0.0]
n_atoms_per_mol = length(template_mol) ÷ 3

template_mol = reshape(template_mol,(3,n_atoms_per_mol))
template_radii = [1.6, 1.2]
radii = vcat([template_radii for i in 1:n_mol]...);

flat_realization = get_flat_realization(x_init, template_mol)
MorphoMol.Utilities.state_to_poly(flat_realization, radii, "assets/output/start", n_mol, n_atoms_per_mol)

In [100]:
T = 5.0
ε = 0.01
L = 4
β = 1.0 / T

rs = 1.4
eta = 0.3665
pf = MorphoMol.Energies.get_prefactors(rs, eta)
overlap_slope = 10.0

σ_r = 0.75
σ_t = 0.5

draw_perturbation! = () -> vcat([[σ_r * randn(3); σ_t * randn(3)] for _ in 1:n_mol]...)
#TODO
inner_product = (p) -> (p ⋅ p)


energy(x) = solvation_free_energy(x, template_mol, radii, rs, pf, 0.0, overlap_slope, 1.0)
energy_gradient!(∇E, x) = solvation_free_energy_gradient!(∇E, x, template_mol, radii, rs, pf, overlap_slope)

hmc = MorphoMol.Algorithms.HamiltonianMonteCarlo(energy, energy_gradient!, inner_product, draw_perturbation!, MorphoMol.Algorithms.standard_leapfrog!, β, L, ε)
iterations = 10000
states, accepted_steps = MorphoMol.Algorithms.simulate!(hmc, deepcopy(x_init), iterations);
accepted_steps / iterations

0.8861

In [102]:
for (i, state) in enumerate(states)
    MorphoMol.Utilities.state_to_poly(get_flat_realization(state, template_mol), radii, "assets/output/$(i)", n_mol, n_atoms_per_mol)
end